In [45]:
!pip install beautifulsoup4 -U
!pip install lxml -U
!pip install requests -U
!pip install html5lib -U
!pip install urllib3 -U

     |████████████████████████████████| 102kB 12.1MB/s ta 0:00:01
  Found existing installation: beautifulsoup4 4.7.1
    Uninstalling beautifulsoup4-4.7.1:
      Successfully uninstalled beautifulsoup4-4.7.1
     |████████████████████████████████| 5.8MB 9.0MB/s eta 0:00:01
  Found existing installation: lxml 4.3.1
    Uninstalling lxml-4.3.1:
      Successfully uninstalled lxml-4.3.1
     |████████████████████████████████| 61kB 13.0MB/s eta 0:00:01
  Found existing installation: requests 2.21.0
    Uninstalling requests-2.21.0:
      Successfully uninstalled requests-2.21.0
Requirement already up-to-date: html5lib in /opt/conda/envs/Python36/lib/python3.6/site-packages (1.0.1)
     |████████████████████████████████| 133kB 16.7MB/s eta 0:00:01
ERROR: ibm-cos-sdk-core 2.4.3 has requirement urllib3<1.25,>=1.20, but you'll have urllib3 1.25.6 which is incompatible.
ERROR: botocore 1.12.82 has requirement urllib3<1.25,>=1.20, but you'll have urllib3 1.25.6 which is incompatible.
  Found ex

In [46]:
import html.parser
from bs4 import BeautifulSoup
import requests
import urllib3.request
import pandas as pd
import numpy as np

In [48]:
url1 = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup1 = BeautifulSoup(response, 'lxml')
table1 = soup1.find('div', id = 'mw-content-text')
rows = table1.find_all('tr')
columns = [v.text.replace('\n','') for v in rows[0].find_all('th')]
df = pd.DataFrame(columns=columns)
for i in range(1, len(rows)):
    tds = rows[i].find_all('td')
    if len(tds) == 3:
        values = [tds[0].text, tds[1].text, tds[2].text.replace('\n','')]
    else:
        pass
    df = df.append(pd.Series(values, index = columns), ignore_index=True)

In [49]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [50]:
df['Borough'].replace('Not assigned',np.nan,inplace = True)
df.dropna(subset=['Borough'], axis = 0, inplace=True)
df.reset_index(drop=True,inplace=True)
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [51]:
df_grouped = df.groupby(["Postcode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
df_grouped.tail(20)

,Postcode,Borough,Neighbourhood
83,M6R,West Toronto,"Parkdale, Roncesvalles"
84,M6S,West Toronto,"Runnymede, Swansea"
85,M7A,Queen's Park,Not assigned
86,M7R,Mississauga,Canada Post Gateway Processing Centre
87,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern
88,M8V,Etobicoke,"Humber Bay Shores, Mimico South, New Toronto"
89,M8W,Etobicoke,"Alderwood, Long Branch"
90,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
91,M8Y,Etobicoke,"Humber Bay, King's Mill Park, Kingsway Park So..."
92,M8Z,Etobicoke,"Kingsway Park South West, Mimico NW, The Queen..."


In [52]:
for index, row in df_grouped.iterrows():
    if row["Neighbourhood"] == "Not assigned":
        row["Neighbourhood"] = row["Borough"]
        
df_grouped.tail(20)

,Postcode,Borough,Neighbourhood
83,M6R,West Toronto,"Parkdale, Roncesvalles"
84,M6S,West Toronto,"Runnymede, Swansea"
85,M7A,Queen's Park,Queen's Park
86,M7R,Mississauga,Canada Post Gateway Processing Centre
87,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern
88,M8V,Etobicoke,"Humber Bay Shores, Mimico South, New Toronto"
89,M8W,Etobicoke,"Alderwood, Long Branch"
90,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
91,M8Y,Etobicoke,"Humber Bay, King's Mill Park, Kingsway Park So..."
92,M8Z,Etobicoke,"Kingsway Park South West, Mimico NW, The Queen..."


In [53]:
df_grouped.shape
!wget -q -O 'toranto_coord.csv' http://cocl.us/Geospatial_data
print('Data downloaded!')
toranto_coord = pd.read_csv("toranto_coord.csv")
toranto_coord.head()
toranto_coord.rename(columns={"Postal Code": "Postcode"}, inplace=True)
toranto_coord.head()

Data downloaded!


,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [54]:
toranto_tab = df_grouped.merge(toranto_coord, on="Postcode", how="left")
toranto_tab.head()
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(toranto_tab['Borough'].unique()),
        toranto_tab.shape[0]))

The dataframe has 11 boroughs and 103 neighborhoods.
